In [1]:
# Snowpark Libs
from snowflake.snowpark.version import VERSION
from snowflake.snowpark.session import Session
import snowflake.snowpark.functions as F
import snowflake.snowpark.types as T
from snowflake.snowpark.functions import col,to_date, udf
# Data Science Libs
import pandas as pd
import os
import dotenv
from pathlib import Path
from datetime import datetime
import numpy as np
import datetime
import pandas as pd



# Laod packages

import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from joblib import Parallel, delayed

#If there are any certificates issues, please run the following command: /Applications/Python\ 3.9/Install\ Certificates.command

In [2]:
# Read config parser .ini file with your connection information

Path = r"/Users/chitrakumarsaichv/Library/CloudStorage/OneDrive-kochind.com/Desktop/Projects/purchasing_anomaly/.env"
dotenv.load_dotenv(Path)

# SSO - KochID
connection_parms = {
    "account": os.getenv('account'),
    "user": os.getenv('email'),
    "authenticator": os.getenv('authenticator'),
    "database": os.getenv('database'),
    "schema": os.getenv('schema'),
    "warehouse": os.getenv('warehouse'),"role": os.getenv('role')
} 


# Create Snowflake Session object
session = Session.builder.configs(connection_parms).create()
session.sql_simplifier_enabled = True

snowflake_environment = session.sql('SELECT current_user(), current_version()').collect()
snowpark_version = VERSION

Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...
Going to open: https://auth.kochid.com/idp/SSO.saml2?SAMLRequest=nZLBbuIwEIZfJfKeYyehrYpFqAKIXbq0oAb2wM2NDXhx7KzHIe3br5OA1D20h5V8sOyZ%2Bf6Zf0YPb6UKzsKCNDpFMY5QIHRhuNSHFG038%2FAeBeCY5kwZLVL0LgA9jEfASlXRrHZH%2FSL%2B1AJc4AtpoO1HimqrqWEggWpWCqCuoHn2tKQJjigDENZ5HLqkcJCedXSuooQ0TYObATb2QJIoikg0JD6qDfmGPiCqrxmVNc4URl1T3nxPnyBiEt20CB%2FhCetL4kTqfgRfUV77IKA%2FNpt1uF7lGxRk1%2B6mRkNdCpsLe5aF2L4sewHgFUh9ln6oXiXHNYSCgQsTzBrAoE2zV%2BwkClNWtfPVsb%2BRveBEmYP0M1vMUlSdJH%2BcuqGYv8rMxczcuBU7Pf68e5bbSdYsf%2Bv9sFFPejlh33e7bYGCX1eHk9bhBUAtFrr11fmnKLkNo4E%2FmySmSUJvIzy4u9%2BhYOZ9lZq5LvMqnnnT8ckUR8k7bZJXJM9XuLUlQf1q0I5gx%2F%2Fb8Ih8rHLZtmdvwGK2NkoW78Hc2JK5z%2F2Jcdy9SB7uu1AqSiZVxrkVAN4npUwztYI5v9TO1gKRcU%2F9d63HfwE%3D&RelayState=ver%3A1-hint%3A759822675599390-ETMsDgAAAZVd4%2BGAABRB

You might have more than one threads sharing the Session object trying to update sql_simplifier_enabled. Updating this while other tasks are running can potentially cause unexpected behavior. Please update the session configuration before starting the threads.


In [3]:
def get_data(session: Session) -> pd.DataFrame:
    # Load data from Snowflake
    """
    This function loads data from the table PURCHASES in the Snowflake database

    Args:
        session (Session): snowflake session object

    Returns:
        pd.DataFrame: returns the data from the table PURCHASES
    """
    with open("/Users/chitrakumarsaichv/Library/CloudStorage/OneDrive-kochind.com/Desktop/Projects/purchasing_anomaly/data/purchasing_anomaly.sql", "r") as file:
        query = file.read()
    df = pd.read_sql(query, session.connection)
    return df

In [4]:
df = get_data(session)

/var/folders/d6/zg1ft6rd1s7122t4rmw9kkpr0000gn/T/ipykernel_42620/3770731770.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, session.connection)


In [5]:
df.head()

,Purchase Doc Date,Purchase Doc Month,Purchase Doc Quarter,Purchase Doc Year,Requested Delivery Date,Vendor ID,Vendor NM,Plant Desc,Purchase Doc ID,Purchase Doc Item ID,...,Work Order No,Network ID,Project ID,Currency Desc,Long Text,Purchase Doc Item Ct,PO Qty,PO Item Net Order Value USD,User Desc,User Manager Email
0,2023-12-11,2023-12,2023 QTR-4,2023,2023-12-12,8010008932,STAPLES ADVANTAGE,Seaford (US) MFG,2000093413,10,...,,,NaN,US Dollar,None,1,1.0,698.690,"Ennis, Jeffery",justin.petrosky@invista.com
1,2023-12-11,2023-12,2023 QTR-4,2023,2023-12-15,8010005836,FASTENAL CANADA LTD,Maitland (CA) MFG,5501699665,20,...,000901093086,,NaN,Canadian Dollar,20 x 13 x 11 Red/Black Matte Finish 32-Pocket ...,1,1.0,137.472,None,None
2,2024-04-02,2024-04,2024 QTR-2,2024,2024-04-09,8010000984,GRAINGER INC,Victoria (US) MFG,2000104892,30,...,,,NaN,US Dollar,None,1,12.0,31.920,"Girdy, Brenton",kyle.hutchins@invista.com
3,2024-06-06,2024-06,2024 QTR-2,2024,2024-06-10,8010000984,GRAINGER INC,Camden Polymer (US) MFG,5501753321,20,...,000901227921,,NaN,US Dollar,TK104119145T Benchtop Turntable Product Type B...,1,1.0,135.600,"Phillips, Tracy",jr.west@invista.com
4,2024-06-06,2024-06,2024 QTR-2,2024,2024-06-14,8010099066,AMAZON.COM SERVICES INC,Seaford (US) MFG,2000111985,20,...,,,NaN,US Dollar,None,1,1.0,15.300,"Townsend, Mack",antonio.d.williams@invista.com


In [6]:
# Define stopwords manually (Scikit-learn's stopword list is English only)
stopwords = set([
    'in', 'height', 'size', 'length', 'mil', 'width', 'ft', 
    'lb', 'overall', 'yes', 'can', 'for', 'per', 'and',
    'the', 'wt', 'oz', 'of', 'or', 'a', 'do', 'an',
    'any', 'am', 'all', 'after', 'again', 'above', 'by',
    'should', 'did', 'each', 'had', 'from', 'her', 'him',
    'but', 'by', 'below', 'between', 'at', 'such', 'some',
    'same', 'out', 'with', 'and', 'is', 'so', 'staples'
])

def process_data(df: pd.DataFrame) -> pd.DataFrame:
    """Filter dataset to keep only Purchase Orders starting with '55' or '20'."""
    df = df[df['Purchase Doc ID'].astype(str).str[:2].isin(['55', '20'])].reset_index(drop=True)
    return df

def get_date_features(df: pd.DataFrame, column_name: str):
    """Extract year and month from a date column."""
    df[column_name] = pd.to_datetime(df[column_name], errors='coerce')
    df[f'{column_name} Year'] = df[column_name].dt.year
    df[f'{column_name} Month'] = df[column_name].dt.month
    return df

def clean_text(text_series: pd.Series) -> pd.Series:
    """Cleans text: removes stopwords, digits, and short words."""
    text_series_clean = text_series.str.lower()
    text_series_clean = text_series_clean.str.replace(r'[^a-z\s]', '', regex=True)  # Remove non-alphabetic characters
    text_series_clean = text_series_clean.apply(lambda x: ' '.join([w for w in x.split() if len(w) > 2 and w not in stopwords]))
    return text_series_clean

def get_unique_words(df: pd.DataFrame, clean_text_col: str) -> list:
    """Returns unique words from the clean text column."""
    all_words = df[clean_text_col].str.split().explode().unique()
    return list(set(all_words))

def first_element(text: str) -> str:
    """Extracts the first meaningful part of a text (for vendor-specific processing)."""
    return text.split(', ')[0].split(' - ')[0].split('---')[0] if isinstance(text, str) else ''

def get_short_text_by_vendor(df: pd.DataFrame) -> pd.DataFrame:
    """Extracts the relevant short text for each vendor."""
    df['text_clean'] = df['Purchase Document Item Short Text'].copy()  # Default to short text

    for i in df.index:
        long_text = df.loc[i, 'Long Text']  # Use the correct column name

        if pd.isna(long_text):  # If Long Text is NaN, use Short Text
            df.loc[i, 'text_clean'] = df.loc[i, 'Purchase Document Item Short Text']
        elif df.loc[i, 'Vendor NM'] == 'GRAINGER INC':
            df.loc[i, 'text_clean'] = first_element(long_text)
        elif df.loc[i, 'Vendor NM'] == 'AMAZON.COM SERVICES INC':
            df.loc[i, 'text_clean'] = str(long_text).split(', ')[0]
        elif df.loc[i, 'Vendor NM'] == 'STAPLES ADVANTAGE':
            df.loc[i, 'text_clean'] = str(long_text).split(', ')[0]
            product_text = str(long_text).split(', ')[0]
            df.loc[i, 'text_clean'] = product_text.split('---')[0]
    
    return df


def extract_nouns_tfidf(text_series: pd.Series) -> list:
    """Uses TF-IDF Vectorizer to extract key words for each row (approximate nouns)."""
    vectorizer = TfidfVectorizer(stop_words="english", max_features=min(len(text_series), 1000))  # Adjust feature limit dynamically
    X = vectorizer.fit_transform(text_series)

    # Convert TF-IDF matrix back to text
    feature_names = vectorizer.get_feature_names_out()

    # Extract top words per row
    word_lists = []
    for row in X.toarray():
        top_indices = row.argsort()[-5:][::-1]  # Get top 5 words per row
        top_words = [feature_names[i] for i in top_indices if row[i] > 0]
        word_lists.append(" ".join(top_words))

    return word_lists

def extract_noun_words(df: pd.DataFrame, text_col: str) -> pd.DataFrame:
    """Extracts top words using TF-IDF as a proxy for important words."""
    df['noun_words'] = extract_nouns_tfidf(df[text_col])
    return df

def get_noun_words_reduced(df: pd.DataFrame):
    """Filters clean text to retain only top 5 extracted nouns."""
    df['text_clean'] = df['text_clean'].str.split()
    df['noun_words'] = df['noun_words'].str.split()
    
    df['noun_words_reduced'] = df.apply(lambda row: ' '.join([w for w in row['text_clean'][:5] if w in row['noun_words']]), axis=1)
    
    df['text_clean'] = df['text_clean'].str.join(' ')
    df['noun_words'] = df['noun_words'].str.join(' ')
    
    return df

def text_parallel_clean(array):
    """Parallel processing wrapper for extract_nouns_tfidf."""
    result = Parallel(n_jobs=8, backend="multiprocessing")(delayed(extract_nouns_tfidf)(text) for text in array)
    return result

def series_of_lists_to_list_of_strings(series_of_lists: pd.Series) -> list:
    """Converts lists in a Pandas series to a list of joined strings."""
    return series_of_lists.apply(lambda x: ' '.join(x) if isinstance(x, list) else '')

def clean_cols(df: pd.DataFrame) -> pd.DataFrame:
    """Formats column names for consistency."""
    df.columns = df.columns.str.replace('_', ' ').str.title()
    return df

In [7]:
df = process_data(df)
df = get_short_text_by_vendor(df)
df['text_clean'] = clean_text(df['text_clean'])
df = extract_noun_words(df, "text_clean")
df = get_noun_words_reduced(df)
df = clean_cols(df)

In [ ]:
# Ensure 'Purchase Doc Date' is in datetime format
df['Purchase Doc Date'] = pd.to_datetime(df['Purchase Doc Date'], errors='coerce')

# Sort by 'Purchase Doc Date' in ascending order (oldest to newest)
df = df.sort_values(by='Purchase Doc Date', ascending=True).reset_index(drop=True)

: 

: 

In [ ]:
df[['Purchase Doc Date','Purchase Document Item Short Text', 'Noun Words Reduced', 'Noun Words',"Plant Desc"]].tail(50)

,Purchase Doc Date,Purchase Document Item Short Text,Noun Words Reduced,Noun Words,Plant Desc
61808,2025-03-03,"Expo Whiteboard Care Cleaner, Blue (1752",expo whiteboard cleaner blue,whiteboard expo cleaner blue,Camden Polymer (US) MFG
61809,2025-03-03,TK118858054T Coated Gloves ANSI/ISEA Cut,tkt coated gloves ansiisea cut,ansiisea coated cut gloves tkt,Seaford (US) MFG
61810,2025-03-03,Logitech MK270 Wireless Keyboard And Mou,logitech wireless keyboard,keyboard logitech wireless,Camden Polymer (US) MFG
61811,2025-03-03,TK118858058T Leather Gloves Hand Protect,tkt leather gloves hand protect,protect hand leather gloves tkt,Seaford (US) MFG
61812,2025-03-03,TK118858065T Safety-Rated Flashlight Bat,tkt safetyrated flashlight bat,bat safetyrated flashlight tkt,Seaford (US) MFG
61813,2025-03-03,TK118858059T Safety Knife Application Mu,tkt safety knife application,knife application safety tkt,Seaford (US) MFG
61814,2025-03-03,Black Rifle Coffee Company Freedom Fuel,black coffee,fuel coffee black,Victoria (US) MFG
61815,2025-03-03,Desk Calendar 2025-2025 Calendar -18 Mon,desk calendar calendar,calendar desk,Victoria (US) MFG
61816,2025-03-03,TK118690752T Flexible Coupling For Nomin,tkt flexible coupling,flexible coupling tkt,Camden Polymer (US) MFG
61817,2025-03-03,"Pilot, G2 Premium Gel Roller Pens, Fine",pilot gel roller pens,roller pilot pens fine gel,Camden Polymer (US) MFG


: 

: 

In [8]:
import re
from collections import Counter
# ✅ Convert column to corpus (concatenating all rows)
corpus = " ".join(df["Noun Words Reduced"])

# Function to clean and tokenize text
def clean_and_tokenize(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    tokens = text.split()  # Simple whitespace tokenization
    return tokens

# Tokenize the corpus
tokens = clean_and_tokenize(corpus)

# Get word count using Counter
word_count = Counter(tokens)

# Convert to DataFrame for better readability
word_count_df = pd.DataFrame(word_count.items(), columns=["Word", "Count"]).sort_values(by="Count", ascending=False)

In [24]:
# Define threshold for rare words
threshold = 25

# Identify rare and common words based on the threshold
rare_words_set = set(word_count_df[word_count_df["Count"] <= threshold]["Word"])
common_words_set = set(word_count_df[word_count_df["Count"] > threshold]["Word"])

# Function to classify words in each row
def classify_words(text):
    words = clean_and_tokenize(text)  # Clean and tokenize text
    rare_words = [word for word in words if word in rare_words_set]
    common_words = [word for word in words if word in common_words_set]
    return rare_words, common_words
df["unique_word_count"] = df["rare_words"].apply(lambda x: len(set(x)))
# Apply function to each row
df[["rare_words", "common_words"]] = df["Noun Words"].apply(lambda x: pd.Series(classify_words(x)))

In [ ]:
df.tail()

,Purchase Doc Date,Purchase Doc Month,Purchase Doc Quarter,Purchase Doc Year,Requested Delivery Date,Vendor Id,Vendor Nm,Plant Desc,Purchase Doc Id,Purchase Doc Item Id,...,Po Qty,Po Item Net Order Value Usd,User Desc,User Manager Email,Text Clean,Noun Words,Noun Words Reduced,rare_words,common_words,unique_word_count
61853,2024-06-05,2024-06,2024 QTR-2,2024,2024-06-06,8010005836,FASTENAL CANADA LTD,Maitland (CA) MFG,2000111858,250,...,1.0,20.029,None,None,bottle gal yield grape liquid,liquid bottle gal yield,bottle gal yield liquid,[],"[liquid, bottle, gal, yield]",0
61854,2024-06-05,2024-06,2024 QTR-2,2024,2024-06-12,8010000984,GRAINGER INC,Victoria (US) MFG,2000111907,10,...,24.0,42.000,"Girdy, Brenton",kyle.hutchins@invista.com,tkt jobber drill bit dri,dri jobber drill bit tkt,tkt jobber drill bit dri,[],"[dri, jobber, drill, bit, tkt]",0
61855,2024-06-05,2024-06,2024 QTR-2,2024,2024-06-12,8010000984,GRAINGER INC,Victoria (US) MFG,2000111907,120,...,4.0,333.120,"Girdy, Brenton",kyle.hutchins@invista.com,tkt dry wipe wipe type general,wipe dry general type tkt,tkt dry wipe wipe type,[],"[wipe, dry, general, type, tkt]",0
61856,2024-06-05,2024-06,2024 QTR-2,2024,2024-06-10,8010000984,GRAINGER INC,Victoria (US) MFG,2000111903,200,...,12.0,96.240,"Girdy, Brenton",kyle.hutchins@invista.com,tkt coated gloves ansiisea cut,ansiisea coated cut gloves tkt,tkt coated gloves ansiisea cut,[],"[ansiisea, coated, cut, gloves, tkt]",0
61857,2024-06-05,2024-06,2024 QTR-2,2024,2024-06-12,8010000984,GRAINGER INC,Victoria (US) MFG,2000111907,140,...,2.0,218.420,"Girdy, Brenton",kyle.hutchins@invista.com,tkt product type sorbent pad,sorbent pad product type tkt,tkt product type sorbent pad,[],"[sorbent, pad, product, type, tkt]",0


: 

: 

In [25]:
df[df['unique_word_count'] > 2]

,Purchase Doc Date,Purchase Doc Month,Purchase Doc Quarter,Purchase Doc Year,Requested Delivery Date,Vendor Id,Vendor Nm,Plant Desc,Purchase Doc Id,Purchase Doc Item Id,...,Po Qty,Po Item Net Order Value Usd,User Desc,User Manager Email,Text Clean,Noun Words,Noun Words Reduced,rare_words,common_words,unique_word_count
433,2024-01-15,2024-01,2024 QTR-1,2024,2024-01-19,8010000984,GRAINGER INC,Victoria (US) MFG,5501709046,10,...,2.0,24.20,"Payne, Joseph",deidrick.b.miller@invista.com,tkt tape thickness tape brand nashua common in...,tape imperial warehouse manufacturing common,tape tape,"[imperial, warehouse, manufacturing, common]",[tape],4
448,2023-12-13,2023-12,2023 QTR-4,2023,2023-12-17,8010000984,GRAINGER INC,Victoria (US) MFG,5501700630,10,...,6.0,70.20,"Christensen, Scott",abraham.flores@invista.com,tkt adhesion steel ozin tape brand approved ve...,tape imperial warehouse manufacturing day,tape,"[imperial, warehouse, manufacturing]","[tape, day]",3
1294,2024-03-19,2024-03,2024 QTR-1,2024,2024-03-23,8010000984,GRAINGER INC,Victoria (US) MFG,5501729411,10,...,24.0,290.40,"Kuecker, Michael",john.w.rush@invista.com,tkt tape thickness tape brand nashua common in...,tape imperial warehouse manufacturing common,tape tape,"[imperial, warehouse, manufacturing, common]",[tape],4
1578,2024-04-29,2024-04,2024 QTR-2,2024,2024-05-03,8010000984,GRAINGER INC,Orange (US) MFG,5501741816,10,...,12.0,201.12,None,None,tkt degreaser cleaner chemical product groupin...,ready dilution content recommended percent,,"[ready, recommended, percent]",[],3
1797,2024-12-31,2024-12,2024 QTR-4,2024,2025-01-04,8010000984,GRAINGER INC,Camden Polymer (US) MFG,5501808219,20,...,1.0,2.83,"Amerin, Kimberly",waelmoataz.noureldin@invista.com,tkt hex nut hex type standard hex finish zinc ...,hex material manufacturer dimensional location,hex hex,"[manufacturer, dimensional, location]","[hex, material]",3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61139,2024-10-14,2024-10,2024 QTR-4,2024,2024-10-18,8010000984,GRAINGER INC,Victoria (US) MFG,5501788340,20,...,1.0,5.55,"Chumchal, Charles",kyle.hutchins@invista.com,tkt hex nut hex type standard hex finish plain...,hex material manufacturer dimensional location,hex hex,"[manufacturer, dimensional, location]","[hex, material]",3
61434,2024-03-05,2024-03,2024 QTR-1,2024,2024-03-08,8010000984,GRAINGER INC,Victoria (US) MFG,2000101824,10,...,2.0,485.24,"Payne, Joseph",deidrick.b.miller@invista.com,tkt stick electrodes aws classification materi...,ksi stick tensile strength welded,stick,"[tensile, strength, welded]",[stick],3
61549,2024-03-28,2024-03,2024 QTR-1,2024,2024-04-01,8010000984,GRAINGER INC,Orange (US) MFG,5501732362,70,...,1.0,20.71,"Clark, Jack",dylan.broussard@invista.com,tkt rubber boot shoe style knee toe type steel...,footwear guard pair pattern shoe,shoe,"[footwear, guard, pattern]","[pair, shoe]",3
61550,2024-03-28,2024-03,2024 QTR-1,2024,2024-04-01,8010000984,GRAINGER INC,Orange (US) MFG,5501732362,60,...,2.0,41.42,"Clark, Jack",dylan.broussard@invista.com,tkt rubber boot shoe style knee toe type steel...,footwear guard pair pattern shoe,shoe,"[footwear, guard, pattern]","[pair, shoe]",3


In [26]:
word_count_df.head()

,Word,Count
6,tkt,14669
86,black,2682
166,grain,2022
54,hex,1971
55,drive,1767


In [14]:
import re

df.columns = [re.sub(r'[^a-zA-Z0-9]', "_",\
                         col.replace(' ', "_")).upper() for col in df.columns]

In [15]:
df.head()

,PURCHASE_DOC_DATE,PURCHASE_DOC_MONTH,PURCHASE_DOC_QUARTER,PURCHASE_DOC_YEAR,REQUESTED_DELIVERY_DATE,VENDOR_ID,VENDOR_NM,PLANT_DESC,PURCHASE_DOC_ID,PURCHASE_DOC_ITEM_ID,...,PROJECT_ID,CURRENCY_DESC,LONG_TEXT,PURCHASE_DOC_ITEM_CT,PO_QTY,PO_ITEM_NET_ORDER_VALUE_USD,USER_DESC,TEXT_CLEAN,NOUN_WORDS,NOUN_WORDS_REDUCED
0,2023-11-22,2023-11,2023 QTR-4,2023,2023-11-23,8010005836,FASTENAL CANADA LTD,Maitland (CA) MFG,2000091638,70,...,NaN,Canadian Dollar,None,1,4.0,7.554,"Stephenson, Michael",cell duracellreg procellreg,procellreg duracellreg cell,cell duracellreg procellreg
1,2023-11-22,2023-11,2023 QTR-4,2023,2023-11-24,8010100205,AMAZON BUSINESS CANADA,Kingston (CA) MFG,2000091652,30,...,NaN,Canadian Dollar,None,1,1.0,7.952,"Rogers, Carl",jetech camera lens protector iphone,camera protector lens iphone,camera lens protector iphone
2,2023-11-24,2023-11,2023 QTR-4,2023,2023-12-06,8010005836,FASTENAL CANADA LTD,Kingston (CA) MFG,2000091824,10,...,NaN,Canadian Dollar,None,1,8.0,157.974,"Miller, Andrew",medium mtm rugged comfo,mtm medium,medium mtm
3,2023-11-24,2023-11,2023 QTR-4,2023,2023-11-28,8010008932,STAPLES ADVANTAGE,Orange (US) MFG,2000091803,50,...,NaN,US Dollar,None,1,2.0,37.260,"Ross, Leslie",dixie ultra pathways heavyweight paper,pathways heavyweight dixie ultra paper,dixie ultra pathways heavyweight paper
4,2023-11-24,2023-11,2023 QTR-4,2023,2023-12-08,8010005836,FASTENAL CANADA LTD,Kingston (CA) MFG,2000091822,10,...,NaN,Canadian Dollar,None,1,6.0,49.363,"MacCrae, Cole",net fill aerosol equipment,net aerosol,net aerosol


In [16]:
df.columns

Index(['PURCHASE_DOC_DATE', 'PURCHASE_DOC_MONTH', 'PURCHASE_DOC_QUARTER',
       'PURCHASE_DOC_YEAR', 'REQUESTED_DELIVERY_DATE', 'VENDOR_ID',
       'VENDOR_NM', 'PLANT_DESC', 'PURCHASE_DOC_ID', 'PURCHASE_DOC_ITEM_ID',
       'BI_LAST_UPDATED', 'REQUISITIONER_ID', 'CREATED_BY',
       'REQUISITIONER_MASTER', 'PURCHASE_DOCUMENT_ITEM_SHORT_TEXT', 'UOM',
       'ACCOUNT_ASSIGNMENT_CATEGORY_ID', 'MATERIAL_ID', 'MATERIAL_GROUP_ID',
       'MATERIAL_GROUP_DESC', 'SUB_COMMODITY_DESC', 'COMMODITY_DESC',
       'GL_ACCOUNT_ID', 'GL_ACCOUNT_DESC', 'COST_CENTER_ID',
       'COST_CENTER_DESC', 'COST_CENTER_PERSON_RESP',
       'COST_CENTER_CATEGORY_DESC', 'WORK_ORDER_NO', 'NETWORK_ID',
       'PROJECT_ID', 'CURRENCY_DESC', 'LONG_TEXT', 'PURCHASE_DOC_ITEM_CT',
       'PO_QTY', 'PO_ITEM_NET_ORDER_VALUE_USD', 'USER_DESC', 'TEXT_CLEAN',
       'NOUN_WORDS', 'NOUN_WORDS_REDUCED'],
      dtype='object')

In [17]:
def get_word_categories(df: pd.DataFrame, use_annotations: bool=False, df_labels=None) -> pd.DataFrame:
    """Reduced noun_words will be split into common_words and rare_words.
    A unique word count is given to each record.

    Args:
        df (pd.DataFrame): dataframe with NOUN_WORDS_REDUCED column
        use_annotations (bool, optional): When use_annotations=True, then if red_flag = 1,
            then the words in the record are not considered in the corpus.
            Defaults to False.
        df_labels (pd.DataFrame, optional): df_labels data in s3. Defaults to None to pull labels from DynamoDB table.
    """
#     start_date = df['purchase_doc_date'].max() - pd.Timedelta(days=450)
#     df = df[df['purchase_doc_date']>start_date].copy()
#     df = df.merge(df_labels, on='tdname', how='left')

    df['NOUN_WORDS_REDUCED'] = df['NOUN_WORDS_REDUCED'].replace(np.nan, 'blank')
    df['NOUN_WORDS_REDUCED'] = df['NOUN_WORDS_REDUCED'].str.split()

    dff = pd.DataFrame()
    for plant in df['PLANT_DESC'].unique():
        print(f'Processing {plant}')

        df_plant = df[df['PLANT_DESC']==plant].copy()

        all_text = []
        if use_annotations:
            for i in df_plant[df_plant['annotation']!=1]['NOUN_WORDS_REDUCED']:
                all_text.extend(i)
        else:
            for i in df_plant['NOUN_WORDS_REDUCED']:
                all_text.extend(i)
            
        ###################
        text_series = pd.Series(all_text)
        rare_words = list(text_series.value_counts()[text_series.value_counts()<=3].index)
        rare_words_list = []
        common_words_list = []
        unique_count = []
        for ind in df_plant['NOUN_WORDS_REDUCED'].index:
            unique_counter = 0
            rare_words_found = []
            common_words_found = []
            for w in df_plant.loc[ind, 'NOUN_WORDS_REDUCED']:
                if w in rare_words:
                    unique_counter+=1
                    rare_words_found.append(w)
                else:
                    common_words_found.append(w)
            rw = ' '.join(rare_words_found)
            cw = ' '.join(common_words_found)
            rare_words_list.append(rw)
            common_words_list.append(cw)
            unique_count.append(unique_counter)
            
        df_plant['unique_count'] = unique_count
        df_plant['common_words'] = common_words_list
        df_plant['rare_words'] = rare_words_list
        df_plant.columns = df_plant.columns.str.upper()
        df_plant['NOUN_WORDS_REDUCED'] = df_plant['NOUN_WORDS_REDUCED'].apply(lambda x: ' '.join(x))

        dff = pd.concat([dff, df_plant], axis=0)
    return dff


In [18]:
get_word_categories(df)

Processing Maitland (CA)  MFG
Processing Kingston (CA) MFG
Processing Orange (US) MFG
Processing Victoria (US) MFG
Processing Camden Polymer (US) MFG
Processing Longview (US) MFG
Processing Seaford (US) MFG
Processing Houston (US) MFG


,PURCHASE_DOC_DATE,PURCHASE_DOC_MONTH,PURCHASE_DOC_QUARTER,PURCHASE_DOC_YEAR,REQUESTED_DELIVERY_DATE,VENDOR_ID,VENDOR_NM,PLANT_DESC,PURCHASE_DOC_ID,PURCHASE_DOC_ITEM_ID,...,PURCHASE_DOC_ITEM_CT,PO_QTY,PO_ITEM_NET_ORDER_VALUE_USD,USER_DESC,TEXT_CLEAN,NOUN_WORDS,NOUN_WORDS_REDUCED,UNIQUE_COUNT,COMMON_WORDS,RARE_WORDS
0,2023-11-22,2023-11,2023 QTR-4,2023,2023-11-23,8010005836,FASTENAL CANADA LTD,Maitland (CA) MFG,2000091638,70,...,1,4.0,7.554,"Stephenson, Michael",cell duracellreg procellreg,procellreg duracellreg cell,cell duracellreg procellreg,1,cell duracellreg,procellreg
20,2023-11-30,2023-11,2023 QTR-4,2023,2023-12-01,8010005836,FASTENAL CANADA LTD,Maitland (CA) MFG,2000092331,160,...,1,3.0,24.530,None,tube stokolanreg white afterwo,tube white,tube white,1,white,tube
56,2023-12-14,2023-12,2023 QTR-4,2023,2023-12-18,8010005836,FASTENAL CANADA LTD,Maitland (CA) MFG,5501700867,10,...,1,48.0,433.642,"MacDonald, Joanne",merv synthetic fiber,synthetic,synthetic,1,,synthetic
57,2023-12-14,2023-12,2023 QTR-4,2023,2023-12-15,8010005836,FASTENAL CANADA LTD,Maitland (CA) MFG,2000093837,50,...,1,5.0,59.739,None,scape goat winter gloves linin,winter goat scape gloves,scape goat winter gloves,4,,scape goat winter gloves
58,2023-12-14,2023-12,2023 QTR-4,2023,2023-12-15,8010005836,FASTENAL CANADA LTD,Maitland (CA) MFG,2000093837,80,...,1,1.0,5.937,None,white fullgrain goatskin leather scap,scap fullgrain leather goatskin white,white fullgrain goatskin leather scap,4,white,fullgrain goatskin leather scap
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
920,2025-01-15,2025-01,2025 QTR-1,2025,2025-01-17,8010099066,AMAZON.COM SERVICES INC,Houston (US) MFG,2000134989,10,...,1,1.0,69.990,"Purdion Shute, Lexus",purrugs dirt trapper door mat,purrugs,purrugs,1,,purrugs
972,2025-02-05,2025-02,2025 QTR-1,2025,2025-02-12,8010008932,STAPLES ADVANTAGE,Houston (US) MFG,2000137161,10,...,1,1.0,13.990,"Macias, Daisy",engraved plastic sign cus,sign plastic,plastic sign,2,,plastic sign
978,2025-02-07,2025-02,2025 QTR-1,2025,2025-02-14,8010000984,GRAINGER INC,Houston (US) MFG,2000137408,50,...,1,6.0,121.980,"Reid, Ridge",tkt safety glasses series dawso,glasses safety series tkt,tkt safety glasses series,3,tkt,safety glasses series
979,2025-02-07,2025-02,2025 QTR-1,2025,2025-02-10,8010099066,AMAZON.COM SERVICES INC,Houston (US) MFG,2000137389,30,...,1,1.0,14.990,"Reid, Ridge",sandisk extreme pro sdxc uhsi memo,pro,pro,1,,pro


## Making it Snowflake Ready

In [6]:
def purchasing_anomaly(session: Session)  -> str:
    import pandas as pd
    import numpy as np
    from sklearn.feature_extraction.text import TfidfVectorizer
    from snowflake.snowpark import Session
    import re
    from collections import Counter
    from datetime import datetime, timedelta
    def process_data() -> pd.DataFrame:
        query = """
                SELECT 
                    PAM.*, ZM."User Desc" , ZM."User Manager Email"
                FROM 
                    ZINT_PR.UI_ALL.VW_RPT_PURCHASE_ANOMALIES_MODEL AS PAM
                LEFT JOIN 
                    (SELECT "User ID", "User Desc" , "User Manager Email"
                        FROM "ZINT_MASTER"."UI_SEC"."AD Account") 
                    AS ZM ON ZM."User ID" = PAM."Requisitioner Master"
                WHERE "Purchase Doc Date" >= DATEADD(DAY, -450, GETDATE())
            """

        df = pd.read_sql(query, session.connection)
        df = df[df['Purchase Doc ID'].astype(str).str[:2].isin(['55', '20'])].reset_index(drop=True)
        df['Purchase Doc ID'] = pd.to_numeric(df['Purchase Doc ID'], errors='coerce')  # Convert to numeric
        df['Purchase Doc ID'] = df['Purchase Doc ID'].fillna(0).astype(int)  # Fill NaNs with 0
        df['Vendor NM'] = df['Vendor NM'].astype(str)  # Ensure it's a string
        df['Purchase Document Item Short Text'] = df['Purchase Document Item Short Text'].astype(str)
        df['Long Text'] = df['Long Text'].astype(str)
        df['Purchase Document Item Short Text'] = df['Purchase Document Item Short Text'].fillna('')
        df['Long Text'] = df['Long Text'].fillna('')
        df['Vendor NM'] = df['Vendor NM'].fillna('')

        return df

    def get_date_features(df: pd.DataFrame, column_name: str):
        """Extract year and month from a date column."""
        df[column_name] = pd.to_datetime(df[column_name], errors='coerce')
        df[f'{column_name} Year'] = df[column_name].dt.year
        df[f'{column_name} Month'] = df[column_name].dt.month
        return df

    def clean_text(text_series: pd.Series) -> pd.Series:
        """Cleans text: removes stopwords, digits, and short words."""
            # Define stopwords manually (Scikit-learn's stopword list is English only)
        stopwords = set([
            'in', 'height', 'size', 'length', 'mil', 'width', 'ft', 
            'lb', 'overall', 'yes', 'can', 'for', 'per', 'and',
            'the', 'wt', 'oz', 'of', 'or', 'a', 'do', 'an',
            'any', 'am', 'all', 'after', 'again', 'above', 'by',
            'should', 'did', 'each', 'had', 'from', 'her', 'him',
            'but', 'by', 'below', 'between', 'at', 'such', 'some',
            'same', 'out', 'with', 'and', 'is', 'so', 'staples'
        ])
        text_series_clean = text_series.str.lower()
        text_series_clean = text_series_clean.str.replace(r'[^a-z\s]', '', regex=True)  # Remove non-alphabetic characters
        text_series_clean = text_series_clean.apply(lambda x: ' '.join([w for w in x.split() if len(w) > 2 and w not in stopwords]))
        return text_series_clean

    def get_unique_words(df: pd.DataFrame, clean_text_col: str) -> list:
        """Returns unique words from the clean text column."""
        all_words = df[clean_text_col].str.split().explode().unique()
        return list(set(all_words))

    def first_element(text: str) -> str:
        """Extracts the first meaningful part of a text (for vendor-specific processing)."""
        return text.split(', ')[0].split(' - ')[0].split('---')[0] if isinstance(text, str) else ''

    def get_short_text_by_vendor(df: pd.DataFrame) -> pd.DataFrame:
        """Extracts the relevant short text for each vendor."""
        df['text_clean'] = df['Purchase Document Item Short Text'].copy()  # Default to short text

        for i in df.index:
            long_text = df.loc[i, 'Long Text']  # Use the correct column name

            if pd.isna(long_text):  # If Long Text is NaN, use Short Text
                df.loc[i, 'text_clean'] = df.loc[i, 'Purchase Document Item Short Text']
            elif df.loc[i, 'Vendor NM'] == 'GRAINGER INC':
                df.loc[i, 'text_clean'] = first_element(long_text)
            elif df.loc[i, 'Vendor NM'] == 'AMAZON.COM SERVICES INC':
                df.loc[i, 'text_clean'] = str(long_text).split(', ')[0]
            elif df.loc[i, 'Vendor NM'] == 'STAPLES ADVANTAGE':
                df.loc[i, 'text_clean'] = str(long_text).split(', ')[0]
                product_text = str(long_text).split(', ')[0]
                df.loc[i, 'text_clean'] = product_text.split('---')[0]
        
        return df


    def extract_nouns_tfidf(text_series: pd.Series) -> list:
        """Uses TF-IDF Vectorizer to extract key words for each row (approximate nouns)."""
        vectorizer = TfidfVectorizer(stop_words="english", max_features=min(len(text_series), 1000))  # Adjust feature limit dynamically
        X = vectorizer.fit_transform(text_series)

        # Convert TF-IDF matrix back to text
        feature_names = vectorizer.get_feature_names_out()

        # Extract top words per row
        word_lists = []
        for row in X.toarray():
            top_indices = row.argsort()[-5:][::-1]  # Get top 5 words per row
            top_words = [feature_names[i] for i in top_indices if row[i] > 0]
            word_lists.append(" ".join(top_words))

        return word_lists

    def extract_noun_words(df: pd.DataFrame, text_col: str) -> pd.DataFrame:
        """Extracts top words using TF-IDF as a proxy for important words."""
        df['noun_words'] = extract_nouns_tfidf(df[text_col])
        return df

    def get_noun_words_reduced(df: pd.DataFrame):
        """Filters clean text to retain only top 5 extracted nouns."""
        df['text_clean'] = df['text_clean'].str.split()
        df['noun_words'] = df['noun_words'].str.split()
        
        df['noun_words_reduced'] = df.apply(lambda row: ' '.join([w for w in row['text_clean'][:5] if w in row['noun_words']]), axis=1)
        
        df['text_clean'] = df['text_clean'].str.join(' ')
        df['noun_words'] = df['noun_words'].str.join(' ')
        
        return df

    def text_parallel_clean(array):
        """Parallel processing wrapper for extract_nouns_tfidf."""
        result = Parallel(n_jobs=8, backend="multiprocessing")(delayed(extract_nouns_tfidf)(text) for text in array)
        return result

    def series_of_lists_to_list_of_strings(series_of_lists: pd.Series) -> list:
        """Converts lists in a Pandas series to a list of joined strings."""
        return series_of_lists.apply(lambda x: ' '.join(x) if isinstance(x, list) else '')

    def clean_cols(df: pd.DataFrame) -> pd.DataFrame:
        """Formats column names for consistency."""
        df.columns = df.columns.str.replace('_', ' ').str.title()
        return df
    
    def get_word_categories(df: pd.DataFrame, use_annotations: bool=False, df_labels=None) -> pd.DataFrame:
        """Reduced noun_words will be split into common_words and rare_words.
        A unique word count is given to each record.

        Args:
            df (pd.DataFrame): dataframe with NOUN_WORDS_REDUCED column
            use_annotations (bool, optional): When use_annotations=True, then if red_flag = 1,
                then the words in the record are not considered in the corpus.
                Defaults to False.
            df_labels (pd.DataFrame, optional): df_labels data in s3. Defaults to None to pull labels from DynamoDB table.
        """
#     start_date = df['purchase_doc_date'].max() - pd.Timedelta(days=450)
#     df = df[df['purchase_doc_date']>start_date].copy()
#     df = df.merge(df_labels, on='tdname', how='left')
        df.columns = [re.sub(r'[^a-zA-Z0-9]', "_",\
                        col.replace(' ', "_")).upper() for col in df.columns]

        df['NOUN_WORDS_REDUCED'] = df['NOUN_WORDS_REDUCED'].replace(np.nan, 'blank')
        df['NOUN_WORDS_REDUCED'] = df['NOUN_WORDS_REDUCED'].str.split()

        dff = pd.DataFrame()
        for plant in df['PLANT_DESC'].unique():
            print(f'Processing {plant}')

            df_plant = df[df['PLANT_DESC']==plant].copy()

            all_text = []
            if use_annotations:
                for i in df_plant[df_plant['annotation']!=1]['NOUN_WORDS_REDUCED']:
                    all_text.extend(i)
            else:
                for i in df_plant['NOUN_WORDS_REDUCED']:
                    all_text.extend(i)
                
            ###################
            text_series = pd.Series(all_text)
            rare_words = list(text_series.value_counts()[text_series.value_counts()<=3].index)
            rare_words_list = []
            common_words_list = []
            unique_count = []
            for ind in df_plant['NOUN_WORDS_REDUCED'].index:
                unique_counter = 0
                rare_words_found = []
                common_words_found = []
                for w in df_plant.loc[ind, 'NOUN_WORDS_REDUCED']:
                    if w in rare_words:
                        unique_counter+=1
                        rare_words_found.append(w)
                    else:
                        common_words_found.append(w)
                rw = ' '.join(rare_words_found)
                cw = ' '.join(common_words_found)
                rare_words_list.append(rw)
                common_words_list.append(cw)
                unique_count.append(unique_counter)
                
            df_plant['unique_count'] = unique_count
            df_plant['common_words'] = common_words_list
            df_plant['rare_words'] = rare_words_list
            df_plant.columns = df_plant.columns.str.upper()
            df_plant['NOUN_WORDS_REDUCED'] = df_plant['NOUN_WORDS_REDUCED'].apply(lambda x: ' '.join(x))

            dff = pd.concat([dff, df_plant], axis=0)
        return dff
    # Step 2: Extract noun words using basic heuristics
    def extract_nouns_simple(text):
        words = text.split()
        # Assuming words longer than 3 characters are likely nouns
        # You can adjust the condition as per your requirements
        nouns = [word for word in words if len(word) > 3]
        return nouns

    
    df = process_data()
    df = get_short_text_by_vendor(df)
    df['text_clean'] = clean_text(df['text_clean'])
    df = extract_noun_words(df, "text_clean")
    df = get_noun_words_reduced(df)
    df = clean_cols(df)
    df = get_word_categories(df)
    # Ensure 'Purchase Doc Date' is in datetime format
    df['PURCHASE_DOC_DATE'] = pd.to_datetime(df['PURCHASE_DOC_DATE'], errors='coerce')
    df['PURCHASE_DOCUMENT_ITEM_SHORT_TEXT'].replace(r'[^a-z\s]', '', regex=True)
    # Sort by 'Purchase Doc Date' in ascending order (oldest to newest)
    df = df.sort_values(by='PURCHASE_DOC_DATE', ascending=True).reset_index(drop=True)
    df.columns = [re.sub(r'[^a-zA-Z0-9]', "_",\
                        col.replace(' ', "_")).upper() for col in df.columns]
    # Step 1: Clean the text
    df['TEXT_CLEAN'] = df['PURCHASE_DOCUMENT_ITEM_SHORT_TEXT'].str.lower().replace(r'[^a-z\s]', '', regex=True)


        # Function to extract English words from a text
    def extract_english_words(text):
        # Remove non-alphabetical characters and convert to lowercase
        cleaned_text = re.sub(r'[^a-zA-Z\s]', '', text).lower()
        # Split into words and filter out empty strings
        words = [word for word in cleaned_text.split() if word.isalpha()]
        return ' '.join(words) if words else None

    # Step 1: Extract English words from PURCHASE_DOCUMENT_ITEM_SHORT_TEXT
    df['ENGLISH_WORDS'] = df['PURCHASE_DOCUMENT_ITEM_SHORT_TEXT'].apply(extract_english_words)

    # Step 2: Replace empty values in TEXT_CLEAN, NOUN_WORDS, NOUN_WORDS_REDUCED
    # Check for empty values and replace them with the corresponding cleaned text
    df['TEXT_CLEAN'] = df.apply(lambda x: x['ENGLISH_WORDS'] if not x['TEXT_CLEAN'] else x['TEXT_CLEAN'], axis=1)
    df['NOUN_WORDS'] = df.apply(lambda x: [x['ENGLISH_WORDS']] if not x['NOUN_WORDS'] else x['NOUN_WORDS'], axis=1)
    df['NOUN_WORDS_REDUCED'] = df.apply(lambda x: [x['ENGLISH_WORDS']] if not x['NOUN_WORDS_REDUCED'] else x['NOUN_WORDS_REDUCED'], axis=1)

    # Step 3: Convert NOUN_WORDS and NOUN_WORDS_REDUCED from list to comma-separated string
    df['NOUN_WORDS'] = df['NOUN_WORDS'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)
    df['NOUN_WORDS_REDUCED'] = df['NOUN_WORDS_REDUCED'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)
    df.drop(columns=['ENGLISH_WORDS'], inplace=True)




    # Step 1: Flatten all words in NOUN_WORDS_REDUCED and calculate global frequency
    all_words = [word for sublist in df['NOUN_WORDS_REDUCED'].apply(lambda x: x.split(', ')) for word in sublist]
    word_freq = Counter(all_words)

    # Step 2: Calculate the uniqueness of each word
    total_rows = len(df)
    word_uniqueness = {word: total_rows / freq for word, freq in word_freq.items()}

    # Step 3: Replace each row with the top unique words
    def get_top_unique_words(row_words, word_uniqueness, top_n=3):
        # Get uniqueness score for each word in the row
        word_scores = {word: word_uniqueness.get(word, 0) for word in row_words}
        # Sort words by uniqueness (highest first) and select top N unique words
        top_words = sorted(word_scores, key=word_scores.get, reverse=True)[:top_n]
        return ', '.join(top_words) if top_words else 'No unique words'

    # Apply the uniqueness ranking to each row
    df['NOUN_WORDS_REDUCED'] = df['NOUN_WORDS_REDUCED'].apply(
        lambda x: get_top_unique_words(x.split(', '), word_uniqueness) if isinstance(x, str) else x
    )



    # Step 1: Flatten all words in NOUN_WORDS_REDUCED and calculate global frequency
    all_words = [word for sublist in df['NOUN_WORDS_REDUCED'].apply(lambda x: x.split(', ')) for word in sublist]
    word_freq = Counter(all_words)

    # Step 2: Identify the rarest words
    # Rarest words are those with the lowest frequency
    min_freq = min(word_freq.values())
    rare_words = {word for word, freq in word_freq.items() if freq == min_freq}

    # Step 3: Replace each row with the rarest words
    def get_rare_words(row_words, rare_words):
        # Get words in the row that are in the rare words set
        rare_row_words = [word for word in row_words if word in rare_words]
        return ', '.join(rare_row_words) if rare_row_words else 'No rare words'

    # Apply the rarity check to each row
    df['RARE_WORDS'] = df['NOUN_WORDS_REDUCED'].apply(
        lambda x: get_rare_words(x.split(', '), rare_words) if isinstance(x, str) else x
    )

    # Normalize strings and keep only the first 3 words
    df['NOUN_WORDS_REDUCED'] = df['NOUN_WORDS_REDUCED'].str.split()
    df['NOUN_WORDS_REDUCED'] = df['NOUN_WORDS_REDUCED'].apply(lambda x: x[:3]).apply(lambda x: ' '.join(x))
    # df_last_week = df.groupby('PLANT_DESC').apply(lambda x: x.nlargest(7, 'PURCHASE_DOC_DATE')).reset_index(drop=True)
    df['PURCHASE_DOC_DATE'] = pd.to_datetime(df['PURCHASE_DOC_DATE'])



    import re
    from collections import Counter
    # ✅ Convert column to corpus (concatenating all rows)
    corpus = " ".join(df["NOUN_WORDS_REDUCED"])

    # Function to clean and tokenize text
    def clean_and_tokenize(text):
        text = text.lower()  # Convert to lowercase
        text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
        tokens = text.split()  # Simple whitespace tokenization
        return tokens

    # Tokenize the corpus
    tokens = clean_and_tokenize(corpus)

    # Get word count using Counter
    word_count = Counter(tokens)

    # Convert to DataFrame for better readability
    word_count_df = pd.DataFrame(word_count.items(), columns=["Word", "Count"]).sort_values(by="Count", ascending=False)

        # Define threshold for rare words
    threshold = 25

    # Identify rare and common words based on the threshold
    rare_words_set = set(word_count_df[word_count_df["Count"] <= threshold]["Word"])
    common_words_set = set(word_count_df[word_count_df["Count"] > threshold]["Word"])

    # Function to classify words in each row
    def classify_words(text):
        words = clean_and_tokenize(text)  # Clean and tokenize text
        rare_words = [word for word in words if word in rare_words_set]
        common_words = [word for word in words if word in common_words_set]
        return rare_words, common_words

    # Apply function to each row
    df[["RARE_WORDS", "COMMON_WORDS"]] = df["NOUN_WORDS"].apply(lambda x: pd.Series(classify_words(x)))
    df["UNIQUE_COUNT"] = df["RARE_WORDS"].apply(lambda x: len(set(x)))
    df["RARE_WORDS"] = df["RARE_WORDS"].apply(lambda x: ", ".join(x) if x else "")
    df["COMMON_WORDS"] = df["COMMON_WORDS"].apply(lambda x: ", ".join(x) if x else "")
    session.create_dataframe(df).write.mode("overwrite").save_as_table("PURCHASES_ANOMALY_RESULTS")
    query_week = """
            SELECT *
            FROM (
                SELECT *,
                    ROW_NUMBER() OVER (PARTITION BY PLANT_DESC ORDER BY PURCHASE_DOC_DATE DESC) AS row_num
                FROM DS_D_WORKSPACE.PURCHASING_ANOMALY.PURCHASES_ANOMALY_RESULTS
                WHERE PURCHASE_DOC_DATE >= DATEADD(week, -1, CURRENT_DATE())
            ) subquery
            WHERE row_num <= 7
            ORDER BY PLANT_DESC, PURCHASE_DOC_DATE ASC;"""

    # Convert all other columns to string
    grouped_last_week = session.sql(query_week).to_pandas()
    grouped_last_week.drop(columns=['ROW_NUM'], inplace=True)
    session.create_dataframe(grouped_last_week).write.mode("overwrite").save_as_table("PURCHASES_ANOMALY_RESULTS_LAST_WEEK")
    
    return "PROCESSING DONE"
    

In [7]:
Database = "DS_D_WORKSPACE"
Schema= "PURCHASING_ANOMALY"  # purchasing anomaly schema
session.use_database(Database)
session.use_schema(Schema)

session.sproc.register(func=purchasing_anomaly,
                       name='purchasing_anomaly',
                       stage_location='@STAGE_PURCHASING_ANOMALY',
                       packages=['snowflake-snowpark-python','pandas','numpy','snowflake-ml-python', 'scikit-learn'],
                       is_permanent=True,
                       replace=True,
                       session=session,
                       execute_as='caller')

The version of package 'snowflake-snowpark-python' in the local environment is 1.28.0, which does not fit the criteria for the requirement 'snowflake-snowpark-python'. Your UDF might not work when the package version is different between the server and your local environment.


In [8]:
session.call('PURCHASING_ANOMALY')

'PROCESSING DONE'

## Scheduling DAG

In [41]:
from snowflake.core import Root
from snowflake.core.task import StoredProcedureCall,Cron
from snowflake.core.task.dagv1 import DAG, DAGTask, DAGOperation
from datetime import timedelta

In [42]:
database = "DS_D_WORKSPACE"
warehouse = "DS_D_DEFAULT_WH"
schema = "PURCHASING_ANOMALY"

In [43]:
def processing_pruchasing_anomaly(session: Session) -> None:
    session.call('PURCHASING_ANOMALY')

In [44]:
root = Root(session)
dag_name = "PURCHASING_ANOMALY_DAG"

dag = DAG(dag_name,
          schedule=Cron("0 12 * * 1", "America/Chicago"), warehouse=warehouse
          )

In [45]:
with dag:
    dag_task1 = DAGTask("PURCHASING_ANOMALY_DAG", StoredProcedureCall(processing_pruchasing_anomaly,
                                                                  stage_location='@STAGE_PURCHASING_ANOMALY',
                                                                  packages=['snowflake-snowpark-python','pandas','numpy','snowflake-ml-python', 'scikit-learn']), warehouse=warehouse)
    
    dag_task1
    
root_schema = root.databases[database].schemas[schema]
dag_op = DAGOperation(root_schema)
dag_op.deploy(dag, mode="orreplace")

The version of package 'snowflake-snowpark-python' in the local environment is 1.27.0, which does not fit the criteria for the requirement 'snowflake-snowpark-python'. Your UDF might not work when the package version is different between the server and your local environment.
The version of package 'snowflake-ml-python' in the local environment is 1.7.4, which does not fit the criteria for the requirement 'snowflake-ml-python'. Your UDF might not work when the package version is different between the server and your local environment.


## RANDOM TEST

In [81]:
def purchasing_anomaly()  -> str:
    import pandas as pd
    import numpy as np
    from sklearn.feature_extraction.text import TfidfVectorizer
    from snowflake.snowpark import Session
    import re

    def process_data() -> pd.DataFrame:
        query = """
    SELECT 
        PAM.*, ZM."User Desc" , ZM."User Manager Email"
    FROM 
        ZINT_PR.UI_ALL.VW_RPT_PURCHASE_ANOMALIES_MODEL AS PAM
    LEFT JOIN 
        (SELECT "User ID", "User Desc" , "User Manager Email"
            FROM "ZINT_MASTER"."UI_SEC"."AD Account") 
        AS ZM ON ZM."User ID" = PAM."Requisitioner Master"
    WHERE "Purchase Doc Date" >= DATEADD(DAY, -450, GETDATE())
"""

        df = pd.read_sql(query, session.connection)
        df = df[df['Purchase Doc ID'].astype(str).str[:2].isin(['55', '20'])].reset_index(drop=True)
        df['Purchase Doc ID'] = pd.to_numeric(df['Purchase Doc ID'], errors='coerce')  # Convert to numeric
        df['Purchase Doc ID'] = df['Purchase Doc ID'].fillna(0).astype(int)  # Fill NaNs with 0
        df['Vendor NM'] = df['Vendor NM'].astype(str)  # Ensure it's a string
        df['Purchase Document Item Short Text'] = df['Purchase Document Item Short Text'].astype(str)
        df['Long Text'] = df['Long Text'].astype(str)
        return df

    def get_date_features(df: pd.DataFrame, column_name: str):
        """Extract year and month from a date column."""
        df[column_name] = pd.to_datetime(df[column_name], errors='coerce')
        df[f'{column_name} Year'] = df[column_name].dt.year
        df[f'{column_name} Month'] = df[column_name].dt.month
        return df

    def clean_text(text_series: pd.Series) -> pd.Series:
        """Cleans text: removes stopwords, digits, and short words."""
            # Define stopwords manually (Scikit-learn's stopword list is English only)
        stopwords = set([
            'in', 'height', 'size', 'length', 'mil', 'width', 'ft', 
            'lb', 'overall', 'yes', 'can', 'for', 'per', 'and',
            'the', 'wt', 'oz', 'of', 'or', 'a', 'do', 'an',
            'any', 'am', 'all', 'after', 'again', 'above', 'by',
            'should', 'did', 'each', 'had', 'from', 'her', 'him',
            'but', 'by', 'below', 'between', 'at', 'such', 'some',
            'same', 'out', 'with', 'and', 'is', 'so', 'staples'
        ])
        text_series_clean = text_series.str.lower()
        text_series_clean = text_series_clean.str.replace(r'[^a-z\s]', '', regex=True)  # Remove non-alphabetic characters
        text_series_clean = text_series_clean.apply(lambda x: ' '.join([w for w in x.split() if len(w) > 2 and w not in stopwords]))
        return text_series_clean

    def get_unique_words(df: pd.DataFrame, clean_text_col: str) -> list:
        """Returns unique words from the clean text column."""
        all_words = df[clean_text_col].str.split().explode().unique()
        return list(set(all_words))

    def first_element(text: str) -> str:
        """Extracts the first meaningful part of a text (for vendor-specific processing)."""
        return text.split(', ')[0].split(' - ')[0].split('---')[0] if isinstance(text, str) else ''

    def get_short_text_by_vendor(df: pd.DataFrame) -> pd.DataFrame:
        """Extracts the relevant short text for each vendor."""
        df['text_clean'] = df['Purchase Document Item Short Text'].copy()  # Default to short text

        for i in df.index:
            long_text = df.loc[i, 'Long Text']  # Use the correct column name

            if pd.isna(long_text):  # If Long Text is NaN, use Short Text
                df.loc[i, 'text_clean'] = df.loc[i, 'Purchase Document Item Short Text']
            elif df.loc[i, 'Vendor NM'] == 'GRAINGER INC':
                df.loc[i, 'text_clean'] = first_element(long_text)
            elif df.loc[i, 'Vendor NM'] == 'AMAZON.COM SERVICES INC':
                df.loc[i, 'text_clean'] = str(long_text).split(', ')[0]
            elif df.loc[i, 'Vendor NM'] == 'STAPLES ADVANTAGE':
                df.loc[i, 'text_clean'] = str(long_text).split(', ')[0]
                product_text = str(long_text).split(', ')[0]
                df.loc[i, 'text_clean'] = product_text.split('---')[0]
        
        return df


    def extract_nouns_tfidf(text_series: pd.Series) -> list:
        """Uses TF-IDF Vectorizer to extract key words for each row (approximate nouns)."""
        vectorizer = TfidfVectorizer(stop_words="english", max_features=min(len(text_series), 1000))  # Adjust feature limit dynamically
        X = vectorizer.fit_transform(text_series)

        # Convert TF-IDF matrix back to text
        feature_names = vectorizer.get_feature_names_out()

        # Extract top words per row
        word_lists = []
        for row in X.toarray():
            top_indices = row.argsort()[-5:][::-1]  # Get top 5 words per row
            top_words = [feature_names[i] for i in top_indices if row[i] > 0]
            word_lists.append(" ".join(top_words))

        return word_lists

    def extract_noun_words(df: pd.DataFrame, text_col: str) -> pd.DataFrame:
        """Extracts top words using TF-IDF as a proxy for important words."""
        df['noun_words'] = extract_nouns_tfidf(df[text_col])
        return df

    def get_noun_words_reduced(df: pd.DataFrame):
        """Filters clean text to retain only top 5 extracted nouns."""
        df['text_clean'] = df['text_clean'].str.split()
        df['noun_words'] = df['noun_words'].str.split()
        
        df['noun_words_reduced'] = df.apply(lambda row: ' '.join([w for w in row['text_clean'][:5] if w in row['noun_words']]), axis=1)
        
        df['text_clean'] = df['text_clean'].str.join(' ')
        df['noun_words'] = df['noun_words'].str.join(' ')
        
        return df

    def text_parallel_clean(array):
        """Parallel processing wrapper for extract_nouns_tfidf."""
        result = Parallel(n_jobs=8, backend="multiprocessing")(delayed(extract_nouns_tfidf)(text) for text in array)
        return result

    def series_of_lists_to_list_of_strings(series_of_lists: pd.Series) -> list:
        """Converts lists in a Pandas series to a list of joined strings."""
        return series_of_lists.apply(lambda x: ' '.join(x) if isinstance(x, list) else '')

    def clean_cols(df: pd.DataFrame) -> pd.DataFrame:
        """Formats column names for consistency."""
        df.columns = df.columns.str.replace('_', ' ').str.title()
        return df
    
    def get_word_categories(df: pd.DataFrame, use_annotations: bool=False, df_labels=None) -> pd.DataFrame:
        """Reduced noun_words will be split into common_words and rare_words.
        A unique word count is given to each record.

        Args:
            df (pd.DataFrame): dataframe with NOUN_WORDS_REDUCED column
            use_annotations (bool, optional): When use_annotations=True, then if red_flag = 1,
                then the words in the record are not considered in the corpus.
                Defaults to False.
            df_labels (pd.DataFrame, optional): df_labels data in s3. Defaults to None to pull labels from DynamoDB table.
        """
#     start_date = df['purchase_doc_date'].max() - pd.Timedelta(days=450)
#     df = df[df['purchase_doc_date']>start_date].copy()
#     df = df.merge(df_labels, on='tdname', how='left')
        df.columns = [re.sub(r'[^a-zA-Z0-9]', "_",\
                        col.replace(' ', "_")).upper() for col in df.columns]

        df['NOUN_WORDS_REDUCED'] = df['NOUN_WORDS_REDUCED'].replace(np.nan, 'blank')
        df['NOUN_WORDS_REDUCED'] = df['NOUN_WORDS_REDUCED'].str.split()

        dff = pd.DataFrame()
        for plant in df['PLANT_DESC'].unique():
            print(f'Processing {plant}')

            df_plant = df[df['PLANT_DESC']==plant].copy()

            all_text = []
            if use_annotations:
                for i in df_plant[df_plant['annotation']!=1]['NOUN_WORDS_REDUCED']:
                    all_text.extend(i)
            else:
                for i in df_plant['NOUN_WORDS_REDUCED']:
                    all_text.extend(i)
                
            ###################
            text_series = pd.Series(all_text)
            rare_words = list(text_series.value_counts()[text_series.value_counts()<=3].index)
            rare_words_list = []
            common_words_list = []
            unique_count = []
            for ind in df_plant['NOUN_WORDS_REDUCED'].index:
                unique_counter = 0
                rare_words_found = []
                common_words_found = []
                for w in df_plant.loc[ind, 'NOUN_WORDS_REDUCED']:
                    if w in rare_words:
                        unique_counter+=1
                        rare_words_found.append(w)
                    else:
                        common_words_found.append(w)
                rw = ' '.join(rare_words_found)
                cw = ' '.join(common_words_found)
                rare_words_list.append(rw)
                common_words_list.append(cw)
                unique_count.append(unique_counter)
                
            df_plant['unique_count'] = unique_count
            df_plant['common_words'] = common_words_list
            df_plant['rare_words'] = rare_words_list
            df_plant.columns = df_plant.columns.str.upper()
            df_plant['NOUN_WORDS_REDUCED'] = df_plant['NOUN_WORDS_REDUCED'].apply(lambda x: ' '.join(x))

            dff = pd.concat([dff, df_plant], axis=0)
        return dff
    
    
    df = process_data()
    df = get_short_text_by_vendor(df)
    df['text_clean'] = clean_text(df['text_clean'])
    df = extract_noun_words(df, "text_clean")
    df = get_noun_words_reduced(df)
    df = clean_cols(df)
    print(df.columns.tolist())
    df = get_word_categories(df)
    df.columns = [re.sub(r'[^a-zA-Z0-9]', "_",\
                    col.replace(' ', "_")).upper() for col in df.columns]
    # Ensure 'Purchase Doc Date' is in datetime format
    df['PURCHASE_DOC_DATE'] = pd.to_datetime(df['PURCHASE_DOC_DATE'], errors='coerce')

    # Sort by 'Purchase Doc Date' in ascending order (oldest to newest)
    df = df.sort_values(by='PURCHASE_DOC_DATE', ascending=True).reset_index(drop=True)
    df.columns = [re.sub(r'[^a-zA-Z0-9]', "_",\
                        col.replace(' ', "_")).upper() for col in df.columns]
    session.create_dataframe(df).write.mode("overwrite").save_as_table("PURCHASES_ANOMALY_RESULTS")
    df = df.astype(str)
    df['PURCHASE_DOC_DATE'] = pd.to_datetime(df['PURCHASE_DOC_DATE'], errors='coerce')
    df_last_week = df.groupby('PLANT_DESC').apply(lambda x: x.nlargest(7, 'PURCHASE_DOC_DATE')).reset_index(drop=True).fillna("")
    session.create_dataframe(df_last_week).write.mode("append").save_as_table("PURCHASES_ANOMALY_RESULTS_LAST_WEEK")
    
    return "PROCESSING DONE"
    

In [82]:
purchasing_anomaly()

/var/folders/d6/zg1ft6rd1s7122t4rmw9kkpr0000gn/T/ipykernel_10934/3298618834.py:21: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, session.connection)


['Purchase Doc Date', 'Purchase Doc Month', 'Purchase Doc Quarter', 'Purchase Doc Year', 'Requested Delivery Date', 'Vendor Id', 'Vendor Nm', 'Plant Desc', 'Purchase Doc Id', 'Purchase Doc Item Id', 'Bi Last Updated', 'Requisitioner Id', 'Created By', 'Requisitioner Master', 'Purchase Document Item Short Text', 'Uom', 'Account Assignment Category Id', 'Material Id', 'Material Group Id', 'Material Group Desc', 'Sub Commodity Desc', 'Commodity Desc', 'Gl Account Id', 'Gl Account Desc', 'Cost Center Id', 'Cost Center Desc', 'Cost Center Person Resp', 'Cost Center Category Desc', 'Work Order No', 'Network Id', 'Project Id', 'Currency Desc', 'Long Text', 'Purchase Doc Item Ct', 'Po Qty', 'Po Item Net Order Value Usd', 'User Desc', 'User Manager Email', 'Text Clean', 'Noun Words', 'Noun Words Reduced']
Processing Victoria (US) MFG
Processing Kingston (CA) MFG
Processing Camden Polymer (US) MFG
Processing Seaford (US) MFG
Processing Longview (US) MFG
Processing Orange (US) MFG
Processing Mait

/var/folders/d6/zg1ft6rd1s7122t4rmw9kkpr0000gn/T/ipykernel_10934/3298618834.py:218: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_last_week = df.groupby('PLANT_DESC').apply(lambda x: x.nlargest(7, 'PURCHASE_DOC_DATE')).reset_index(drop=True).fillna("")


'PROCESSING DONE'

In [33]:
a, df =purchasing_anomaly()

/var/folders/d6/zg1ft6rd1s7122t4rmw9kkpr0000gn/T/ipykernel_10934/2089985649.py:21: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, session.connection)


Processing Camden Polymer (US) MFG
Processing Victoria (US) MFG
Processing Kingston (CA) MFG
Processing Seaford (US) MFG
Processing Orange (US) MFG
Processing Houston (US) MFG
Processing Maitland (CA)  MFG
Processing Longview (US) MFG


In [67]:

# Normalize strings and keep only the first 3 words
df['PURCHASE_DOCUMENT_ITEM_SHORT_TEXT_sample'] = df['PURCHASE_DOCUMENT_ITEM_SHORT_TEXT'].str.split()

In [73]:
df['PURCHASE_DOCUMENT_ITEM_SHORT_TEXT_sample_test'] = df['PURCHASE_DOCUMENT_ITEM_SHORT_TEXT_sample'].apply(lambda x: x[:3]).apply(lambda x: ' '.join(x))

In [74]:
df[['PURCHASE_DOCUMENT_ITEM_SHORT_TEXT_sample_test']]

,PURCHASE_DOCUMENT_ITEM_SHORT_TEXT_sample_test
0,TK94564287T Primerless Medium-Strength
1,TK94566746T Liquid Storage
5,TK94566744T Facial Tissue
8,TK94564288T AA Battery
9,TK94564289T AAA Battery
...,...
61470,TK118450437T Cutting Fluid
61537,"Hirsh 48""W Single-Pedestal"
61560,Staples Essentials Ergonomic
61574,TK118450763T Hydraulic Quick


In [95]:
def purchasing_anomaly()  -> str:
    import pandas as pd
    import numpy as np
    from sklearn.feature_extraction.text import TfidfVectorizer
    from snowflake.snowpark import Session
    import re
    from collections import Counter
    def process_data(session:Session) -> pd.DataFrame:
        query = """
                SELECT 
                    PAM.*, ZM."User Desc" , ZM."User Manager Email"
                FROM 
                    ZINT_PR.UI_ALL.VW_RPT_PURCHASE_ANOMALIES_MODEL AS PAM
                LEFT JOIN 
                    (SELECT "User ID", "User Desc" , "User Manager Email"
                        FROM "ZINT_MASTER"."UI_SEC"."AD Account") 
                    AS ZM ON ZM."User ID" = PAM."Requisitioner Master"
                WHERE "Purchase Doc Date" >= DATEADD(DAY, -450, GETDATE())
            """
        df = session.sql(query).to_pandas()
        df = pd.read_sql(query, session.connection)
        df = df[df['Purchase Doc ID'].astype(str).str[:2].isin(['55', '20'])].reset_index(drop=True)
        df['Purchase Doc ID'] = pd.to_numeric(df['Purchase Doc ID'], errors='coerce')  # Convert to numeric
        df['Purchase Doc ID'] = df['Purchase Doc ID'].fillna(0).astype(int)  # Fill NaNs with 0
        df['Vendor NM'] = df['Vendor NM'].astype(str)  # Ensure it's a string
        df['Purchase Document Item Short Text'] = df['Purchase Document Item Short Text'].astype(str)
        df['Long Text'] = df['Long Text'].astype(str)
        df['Purchase Document Item Short Text'] = df['Purchase Document Item Short Text'].fillna('')
        df['Long Text'] = df['Long Text'].fillna('')
        df['Vendor NM'] = df['Vendor NM'].fillna('')

        return df

    def get_date_features(df: pd.DataFrame, column_name: str):
        """Extract year and month from a date column."""
        df[column_name] = pd.to_datetime(df[column_name], errors='coerce')
        df[f'{column_name} Year'] = df[column_name].dt.year
        df[f'{column_name} Month'] = df[column_name].dt.month
        return df

    def clean_text(text_series: pd.Series) -> pd.Series:
        """Cleans text: removes stopwords, digits, and short words."""
            # Define stopwords manually (Scikit-learn's stopword list is English only)
        stopwords = set([
            'in', 'height', 'size', 'length', 'mil', 'width', 'ft', 
            'lb', 'overall', 'yes', 'can', 'for', 'per', 'and',
            'the', 'wt', 'oz', 'of', 'or', 'a', 'do', 'an',
            'any', 'am', 'all', 'after', 'again', 'above', 'by',
            'should', 'did', 'each', 'had', 'from', 'her', 'him',
            'but', 'by', 'below', 'between', 'at', 'such', 'some',
            'same', 'out', 'with', 'and', 'is', 'so', 'staples'
        ])
        text_series_clean = text_series.str.lower()
        text_series_clean = text_series_clean.str.replace(r'[^a-z\s]', '', regex=True)  # Remove non-alphabetic characters
        text_series_clean = text_series_clean.apply(lambda x: ' '.join([w for w in x.split() if len(w) > 2 and w not in stopwords]))
        return text_series_clean

    def get_unique_words(df: pd.DataFrame, clean_text_col: str) -> list:
        """Returns unique words from the clean text column."""
        all_words = df[clean_text_col].str.split().explode().unique()
        return list(set(all_words))

    def first_element(text: str) -> str:
        """Extracts the first meaningful part of a text (for vendor-specific processing)."""
        return text.split(', ')[0].split(' - ')[0].split('---')[0] if isinstance(text, str) else ''

    def get_short_text_by_vendor(df: pd.DataFrame) -> pd.DataFrame:
        """Extracts the relevant short text for each vendor."""
        df['text_clean'] = df['Purchase Document Item Short Text'].copy()  # Default to short text

        for i in df.index:
            long_text = df.loc[i, 'Long Text']  # Use the correct column name

            if pd.isna(long_text):  # If Long Text is NaN, use Short Text
                df.loc[i, 'text_clean'] = df.loc[i, 'Purchase Document Item Short Text']
            elif df.loc[i, 'Vendor NM'] == 'GRAINGER INC':
                df.loc[i, 'text_clean'] = first_element(long_text)
            elif df.loc[i, 'Vendor NM'] == 'AMAZON.COM SERVICES INC':
                df.loc[i, 'text_clean'] = str(long_text).split(', ')[0]
            elif df.loc[i, 'Vendor NM'] == 'STAPLES ADVANTAGE':
                df.loc[i, 'text_clean'] = str(long_text).split(', ')[0]
                product_text = str(long_text).split(', ')[0]
                df.loc[i, 'text_clean'] = product_text.split('---')[0]
        
        return df


    def extract_nouns_tfidf(text_series: pd.Series) -> list:
        """Uses TF-IDF Vectorizer to extract key words for each row (approximate nouns)."""
        vectorizer = TfidfVectorizer(stop_words="english", max_features=min(len(text_series), 1000))  # Adjust feature limit dynamically
        X = vectorizer.fit_transform(text_series)

        # Convert TF-IDF matrix back to text
        feature_names = vectorizer.get_feature_names_out()

        # Extract top words per row
        word_lists = []
        for row in X.toarray():
            top_indices = row.argsort()[-5:][::-1]  # Get top 5 words per row
            top_words = [feature_names[i] for i in top_indices if row[i] > 0]
            word_lists.append(" ".join(top_words))

        return word_lists

    def extract_noun_words(df: pd.DataFrame, text_col: str) -> pd.DataFrame:
        """Extracts top words using TF-IDF as a proxy for important words."""
        df['noun_words'] = extract_nouns_tfidf(df[text_col])
        return df

    def get_noun_words_reduced(df: pd.DataFrame):
        """Filters clean text to retain only top 5 extracted nouns."""
        df['text_clean'] = df['text_clean'].str.split()
        df['noun_words'] = df['noun_words'].str.split()
        
        df['noun_words_reduced'] = df.apply(lambda row: ' '.join([w for w in row['text_clean'][:5] if w in row['noun_words']]), axis=1)
        
        df['text_clean'] = df['text_clean'].str.join(' ')
        df['noun_words'] = df['noun_words'].str.join(' ')
        
        return df

    def text_parallel_clean(array):
        """Parallel processing wrapper for extract_nouns_tfidf."""
        result = Parallel(n_jobs=8, backend="multiprocessing")(delayed(extract_nouns_tfidf)(text) for text in array)
        return result

    def series_of_lists_to_list_of_strings(series_of_lists: pd.Series) -> list:
        """Converts lists in a Pandas series to a list of joined strings."""
        return series_of_lists.apply(lambda x: ' '.join(x) if isinstance(x, list) else '')

    def clean_cols(df: pd.DataFrame) -> pd.DataFrame:
        """Formats column names for consistency."""
        df.columns = df.columns.str.replace('_', ' ').str.title()
        return df
    
    def get_word_categories(df: pd.DataFrame, use_annotations: bool=False, df_labels=None) -> pd.DataFrame:
        """Reduced noun_words will be split into common_words and rare_words.
        A unique word count is given to each record.

        Args:
            df (pd.DataFrame): dataframe with NOUN_WORDS_REDUCED column
            use_annotations (bool, optional): When use_annotations=True, then if red_flag = 1,
                then the words in the record are not considered in the corpus.
                Defaults to False.
            df_labels (pd.DataFrame, optional): df_labels data in s3. Defaults to None to pull labels from DynamoDB table.
        """
#     start_date = df['purchase_doc_date'].max() - pd.Timedelta(days=450)
#     df = df[df['purchase_doc_date']>start_date].copy()
#     df = df.merge(df_labels, on='tdname', how='left')
        df.columns = [re.sub(r'[^a-zA-Z0-9]', "_",\
                        col.replace(' ', "_")).upper() for col in df.columns]

        df['NOUN_WORDS_REDUCED'] = df['NOUN_WORDS_REDUCED'].replace(np.nan, 'blank')
        df['NOUN_WORDS_REDUCED'] = df['NOUN_WORDS_REDUCED'].str.split()

        dff = pd.DataFrame()
        for plant in df['PLANT_DESC'].unique():
            print(f'Processing {plant}')

            df_plant = df[df['PLANT_DESC']==plant].copy()

            all_text = []
            if use_annotations:
                for i in df_plant[df_plant['annotation']!=1]['NOUN_WORDS_REDUCED']:
                    all_text.extend(i)
            else:
                for i in df_plant['NOUN_WORDS_REDUCED']:
                    all_text.extend(i)
                
            ###################
            text_series = pd.Series(all_text)
            rare_words = list(text_series.value_counts()[text_series.value_counts()<=3].index)
            rare_words_list = []
            common_words_list = []
            unique_count = []
            for ind in df_plant['NOUN_WORDS_REDUCED'].index:
                unique_counter = 0
                rare_words_found = []
                common_words_found = []
                for w in df_plant.loc[ind, 'NOUN_WORDS_REDUCED']:
                    if w in rare_words:
                        unique_counter+=1
                        rare_words_found.append(w)
                    else:
                        common_words_found.append(w)
                rw = ' '.join(rare_words_found)
                cw = ' '.join(common_words_found)
                rare_words_list.append(rw)
                common_words_list.append(cw)
                unique_count.append(unique_counter)
                
            df_plant['unique_count'] = unique_count
            df_plant['common_words'] = common_words_list
            df_plant['rare_words'] = rare_words_list
            df_plant.columns = df_plant.columns.str.upper()
            df_plant['NOUN_WORDS_REDUCED'] = df_plant['NOUN_WORDS_REDUCED'].apply(lambda x: ' '.join(x))

            dff = pd.concat([dff, df_plant], axis=0)
        return dff
    # Step 2: Extract noun words using basic heuristics
    def extract_nouns_simple(text):
        words = text.split()
        # Assuming words longer than 3 characters are likely nouns
        # You can adjust the condition as per your requirements
        nouns = [word for word in words if len(word) > 3]
        return nouns

    
    df = process_data(session)
    df = get_short_text_by_vendor(df)
    df['text_clean'] = clean_text(df['text_clean'])
    df = extract_noun_words(df, "text_clean")
    df = get_noun_words_reduced(df)
    df = clean_cols(df)
    df = get_word_categories(df)
    # Ensure 'Purchase Doc Date' is in datetime format
    df['PURCHASE_DOC_DATE'] = pd.to_datetime(df['PURCHASE_DOC_DATE'], errors='coerce')
    df['PURCHASE_DOCUMENT_ITEM_SHORT_TEXT'].replace(r'[^a-z\s]', '', regex=True)
    # Sort by 'Purchase Doc Date' in ascending order (oldest to newest)
    df = df.sort_values(by='PURCHASE_DOC_DATE', ascending=True).reset_index(drop=True)
    df.columns = [re.sub(r'[^a-zA-Z0-9]', "_",\
                        col.replace(' ', "_")).upper() for col in df.columns]
    # Step 1: Clean the text
    df['TEXT_CLEAN'] = df['PURCHASE_DOCUMENT_ITEM_SHORT_TEXT'].str.lower().replace(r'[^a-z\s]', '', regex=True)


        # Function to extract English words from a text
    def extract_english_words(text):
        # Remove non-alphabetical characters and convert to lowercase
        cleaned_text = re.sub(r'[^a-zA-Z\s]', '', text).lower()
        # Split into words and filter out empty strings
        words = [word for word in cleaned_text.split() if word.isalpha()]
        return ' '.join(words) if words else None

    # Step 1: Extract English words from PURCHASE_DOCUMENT_ITEM_SHORT_TEXT
    df['ENGLISH_WORDS'] = df['PURCHASE_DOCUMENT_ITEM_SHORT_TEXT'].apply(extract_english_words)

    # Step 2: Replace empty values in TEXT_CLEAN, NOUN_WORDS, NOUN_WORDS_REDUCED
    # Check for empty values and replace them with the corresponding cleaned text
    df['TEXT_CLEAN'] = df.apply(lambda x: x['ENGLISH_WORDS'] if not x['TEXT_CLEAN'] else x['TEXT_CLEAN'], axis=1)
    df['NOUN_WORDS'] = df.apply(lambda x: [x['ENGLISH_WORDS']] if not x['NOUN_WORDS'] else x['NOUN_WORDS'], axis=1)
    df['NOUN_WORDS_REDUCED'] = df.apply(lambda x: [x['ENGLISH_WORDS']] if not x['NOUN_WORDS_REDUCED'] else x['NOUN_WORDS_REDUCED'], axis=1)

    # Step 3: Convert NOUN_WORDS and NOUN_WORDS_REDUCED from list to comma-separated string
    df['NOUN_WORDS'] = df['NOUN_WORDS'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)
    df['NOUN_WORDS_REDUCED'] = df['NOUN_WORDS_REDUCED'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)
    df.drop(columns=['ENGLISH_WORDS'], inplace=True)




    # Step 1: Flatten all words in NOUN_WORDS_REDUCED and calculate global frequency
    all_words = [word for sublist in df['NOUN_WORDS_REDUCED'].apply(lambda x: x.split(', ')) for word in sublist]
    word_freq = Counter(all_words)

    # Step 2: Calculate the uniqueness of each word
    total_rows = len(df)
    word_uniqueness = {word: total_rows / freq for word, freq in word_freq.items()}

    # Step 3: Replace each row with the top unique words
    def get_top_unique_words(row_words, word_uniqueness, top_n=3):
        # Get uniqueness score for each word in the row
        word_scores = {word: word_uniqueness.get(word, 0) for word in row_words}
        # Sort words by uniqueness (highest first) and select top N unique words
        top_words = sorted(word_scores, key=word_scores.get, reverse=True)[:top_n]
        return ', '.join(top_words) if top_words else 'No unique words'

    # Apply the uniqueness ranking to each row
    df['NOUN_WORDS_REDUCED'] = df['NOUN_WORDS_REDUCED'].apply(
        lambda x: get_top_unique_words(x.split(', '), word_uniqueness) if isinstance(x, str) else x
    )



    # Step 1: Flatten all words in NOUN_WORDS_REDUCED and calculate global frequency
    all_words = [word for sublist in df['NOUN_WORDS_REDUCED'].apply(lambda x: x.split(', ')) for word in sublist]
    word_freq = Counter(all_words)

    # Step 2: Identify the rarest words
    # Rarest words are those with the lowest frequency
    min_freq = min(word_freq.values())
    rare_words = {word for word, freq in word_freq.items() if freq == min_freq}

    # Step 3: Replace each row with the rarest words
    def get_rare_words(row_words, rare_words):
        # Get words in the row that are in the rare words set
        rare_row_words = [word for word in row_words if word in rare_words]
        return ', '.join(rare_row_words) if rare_row_words else 'No rare words'

    # Apply the rarity check to each row
    df['RARE_WORDS'] = df['NOUN_WORDS_REDUCED'].apply(
        lambda x: get_rare_words(x.split(', '), rare_words) if isinstance(x, str) else x
    )

    # Normalize strings and keep only the first 3 words
    df['NOUN_WORDS_REDUCED'] = df['NOUN_WORDS_REDUCED'].str.split()
    df['NOUN_WORDS_REDUCED'] = df['NOUN_WORDS_REDUCED'].apply(lambda x: x[:3]).apply(lambda x: ' '.join(x))


    
    return df
    

In [96]:
df = purchasing_anomaly()

/var/folders/d6/zg1ft6rd1s7122t4rmw9kkpr0000gn/T/ipykernel_10934/980268092.py:21: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, session.connection)


Processing Camden Polymer (US) MFG
Processing Kingston (CA) MFG
Processing Maitland (CA)  MFG
Processing Victoria (US) MFG
Processing Houston (US) MFG
Processing Seaford (US) MFG
Processing Longview (US) MFG
Processing Orange (US) MFG


In [97]:
df

,PURCHASE_DOC_DATE,PURCHASE_DOC_MONTH,PURCHASE_DOC_QUARTER,PURCHASE_DOC_YEAR,REQUESTED_DELIVERY_DATE,VENDOR_ID,VENDOR_NM,PLANT_DESC,PURCHASE_DOC_ID,PURCHASE_DOC_ITEM_ID,...,PO_QTY,PO_ITEM_NET_ORDER_VALUE_USD,USER_DESC,USER_MANAGER_EMAIL,TEXT_CLEAN,NOUN_WORDS,NOUN_WORDS_REDUCED,UNIQUE_COUNT,COMMON_WORDS,RARE_WORDS
0,2023-12-03,2023-12,2023 QTR-4,2023,2023-12-05,8010099066,AMAZON.COM SERVICES INC,Victoria (US) MFG,2000092553,20,...,1.0,123.980,"Brown, Matt",joshua.guilbeaux@invista.com,milwaukee m redlithium xc,milwaukee m redlithium xc,milwaukee m redlithium,0,,No rare words
1,2023-12-03,2023-12,2023 QTR-4,2023,2024-02-01,8010005836,FASTENAL CANADA LTD,Kingston (CA) MFG,2000092546,20,...,3.0,49.384,"MacCrae, Cole",bruce.robertson@invista.com,x chrome imperial rock riverre,imperial rock chrome,chrome imperial rock,0,chrome imperial rock,chrome imperial rock
2,2023-12-03,2023-12,2023 QTR-4,2023,2023-12-07,8010000984,GRAINGER INC,Camden Polymer (US) MFG,2000092540,10,...,2.0,32.940,"Newman, Ronnie",carl.l.glenn@invista.com,tkt battery battery chemistry se,tkt battery battery chemistry se,tkt battery battery,0,,tkt battery battery chemistry se
3,2023-12-03,2023-12,2023 QTR-4,2023,2023-12-03,8010099066,AMAZON.COM SERVICES INC,Victoria (US) MFG,2000092539,10,...,1.0,21.970,"Ramirez, Isaac",james.perez@invista.com,pilot frixion clicker erasable gel pens,pilot frixion clicker erasable gel pens,pilot frixion clicker,0,,No rare words
4,2023-12-03,2023-12,2023 QTR-4,2023,2023-12-07,8010000984,GRAINGER INC,Camden Polymer (US) MFG,2000092540,30,...,2.0,17.460,"Newman, Ronnie",carl.l.glenn@invista.com,tkt aa battery aa high performan,tkt aa battery aa high performan,tkt aa battery,0,,No rare words
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61614,2025-02-24,2025-02,2025 QTR-1,2025,2025-02-28,8010005836,FASTENAL CANADA LTD,Kingston (CA) MFG,2000139222,40,...,7.0,12.341,"Louwerse, James",chris.a.boutilier@invista.com,x xhd degf ptfe ormadusr,ptfe degf,degf ptfe,0,degf ptfe,No rare words
61615,2025-02-24,2025-02,2025 QTR-1,2025,2025-02-24,8010005836,FASTENAL CANADA LTD,Kingston (CA) MFG,2000139192,50,...,1.0,39.564,"Casselman, Christopher",scott.aikman@invista.com,type r heat cutter blade hsgm h,heat cutter blade type,type heat cutter,0,type heat cutter blade,No rare words
61616,2025-02-24,2025-02,2025 QTR-1,2025,2025-02-28,8010008932,STAPLES ADVANTAGE,Houston (US) MFG,2000139226,80,...,1.0,80.170,"Askew, Jeremy",grant.hopkins@invista.com,snickers milk chocolate candy bar,snickers milk chocolate candy bar,snickers milk chocolate,0,,No rare words
61617,2025-02-24,2025-02,2025 QTR-1,2025,2025-03-06,8010099066,AMAZON.COM SERVICES INC,Victoria (US) MFG,2000139195,90,...,1.0,74.600,"Fayed, Caitlyn",ashley.b.awalt@invista.com,aula f wireless mechanical keyboardtr,aula f wireless mechanical keyboardtr,aula f wireless,0,,aula f wireless mechanical keyboardtr


In [ ]:
query = """
        SELECT 
            PAM.*, ZM."User Desc" , ZM."User Manager Email"
        FROM 
            ZINT_PR.UI_ALL.VW_RPT_PURCHASE_ANOMALIES_MODEL AS PAM
        LEFT JOIN 
            (SELECT "User ID", "User Desc" , "User Manager Email"
                FROM "ZINT_MASTER"."UI_SEC"."AD Account") 
            AS ZM ON ZM."User ID" = PAM."Requisitioner Master"
        WHERE "Purchase Doc Date" >= DATEADD(DAY, -450, GETDATE());
    """

In [100]:
df = session.sql(query).to_pandas()